To be fair, a cut should include anyone who is tied with the Nth-place competitor. --> peptidi koji su tied sa Nth-place peptidom su jednako dobri za branching kao i Nth-place peptid pa nema smisla excludeati ih iz branchinga

Thus, Leaderboard should be trimmed down to the “N highest-scoring linear peptides including ties”, which may include more than N peptides. Given a list of peptides Leaderboard, a spectrum Spectrum, and an integer N, define TRIM(Leaderboard, Spectrum, N) as the collection of the top N highest-scoring linear peptides in Leaderboard (including ties) with respect to Spectrum.

In [1]:
def FindAllKmers(peptide,k):
  kmers_list = []
  i = 0
  while i + k - 1 <= len(peptide)-1:
    kmers_list.append(peptide[i:i+k])
    i = i + 1
  return kmers_list

In [2]:
def GenerateAllSubpeptides(peptide):
  subpeptides = []
  for subpeptide_length in range(1,len(peptide)):
    subpeptides.extend(FindAllKmers(peptide,subpeptide_length))
  subpeptides.append(peptide)
  return subpeptides

In [3]:
aminoacids_integer_masses_dict = {'A':71.03711,
'R':156.10111,
'N':114.04293,
'D':115.02694,
'C':103.00919,
'E':129.04259,
'Q':128.05858,
'G':57.02146,
'H':137.05891,
'I':113.08406,
'L':113.08406,
'K':128.09496,
'M':131.04049,
'F':147.06841,
'P':97.05276,
'S':87.03203,
'T':101.04768,
'W':186.07931,
'Y':163.06333,
'V':99.06841}

In [4]:
for key in aminoacids_integer_masses_dict:
  aminoacids_integer_masses_dict[key] = int(aminoacids_integer_masses_dict[key])

In [5]:
def SubpeptideMass(subpeptide):
  mass = 0
  for aminoacid in subpeptide:
    mass = mass + aminoacids_integer_masses_dict[aminoacid]
  return mass

In [6]:
def LinearSpectrum(peptide):
  theoretical_linear_spectrum = [0]
  for subpeptide in GenerateAllSubpeptides(peptide):
    theoretical_linear_spectrum.append(SubpeptideMass(subpeptide))
  return theoretical_linear_spectrum

In [7]:
from collections import Counter

In [8]:
def LinearPeptideScoring(peptide,spectrum): #the score is computed for spectrum against peptide's theoretical spectrum
  peptide_theoretical_spectrum = LinearSpectrum(peptide)
  peptide_theoretical_spectrum_counter_dict = Counter(peptide_theoretical_spectrum)
  spectrum_counter_dict = Counter(spectrum)
  score = 0
  scored_aminoacid_integer_masses = []
  for aminoacid_integer_mass in spectrum:
    if aminoacid_integer_mass not in scored_aminoacid_integer_masses:
      if spectrum_counter_dict[aminoacid_integer_mass] == peptide_theoretical_spectrum_counter_dict[aminoacid_integer_mass]:
        score = score + spectrum_counter_dict[aminoacid_integer_mass]
        scored_aminoacid_integer_masses.append(aminoacid_integer_mass)
      elif spectrum_counter_dict[aminoacid_integer_mass] > peptide_theoretical_spectrum_counter_dict[aminoacid_integer_mass]:
        if peptide_theoretical_spectrum_counter_dict[aminoacid_integer_mass] > 0: #if peptide_theoretical_spectrum_counter_dict[aminoacid_integer_mass] > 0 then there are surpluss occurences of same mass in experimental spectrum
          score = score + peptide_theoretical_spectrum_counter_dict[aminoacid_integer_mass]
          scored_aminoacid_integer_masses.append(aminoacid_integer_mass)
      else: #spectrum_counter_dict[aminoacid_integer_mass] < peptide_theoretical_spectrum_counter_dict[aminoacid_integer_mass] --> there are surpluss occurences of same mass in theoretical spectrum
        if spectrum_counter_dict[aminoacid_integer_mass] > 0:
          score = score + spectrum_counter_dict[aminoacid_integer_mass]
          scored_aminoacid_integer_masses.append(aminoacid_integer_mass)
  return score

In [9]:
def Trim(leaderboard,linear_spectrum,N):
  leaderboard_scores_dict = {}
  for linear_peptide in leaderboard:
    leaderboard_scores_dict.update({linear_peptide:LinearPeptideScoring(linear_peptide,linear_spectrum)})
  sorted_linear_peptides = sorted(leaderboard_scores_dict.keys(), key=leaderboard_scores_dict.get, reverse=True)
  top_n_peptides = sorted_linear_peptides[0:N] #last index is N-1, next index is N
  i = N
  while sorted_linear_peptides[i] == top_n_peptides[N-1]:
    sorted_linear_peptides.append(sorted_linear_peptides[i])
  return top_n_peptides

In [10]:
leaderboard = ['LAST', 'ALST', 'TLLT', 'TQAS']

In [11]:
linear_spectrum = [0, 71, 87, 101, 113, 158, 184, 188, 259, 271, 372]

In [12]:
N = 2

In [13]:
for solution in Trim(leaderboard,linear_spectrum,N):
  print(solution)

LAST
ALST


In [39]:
with open('/content/rosalind_ba4l.txt') as task_file:
  leaderboard = [line.rstrip() for line in task_file]

In [40]:
leaderboard = leaderboard[0]

In [41]:
leaderboard = leaderboard.split(' ')

In [ ]:
leaderboard

In [32]:
with open('/content/rosalind_ba4l.txt') as task_file:
  linear_spectrum = [line.rstrip() for line in task_file]

In [34]:
linear_spectrum = linear_spectrum[0]

In [36]:
linear_spectrum = linear_spectrum.split(' ')

In [37]:
for i in range(len(linear_spectrum)):
  linear_spectrum[i] = int(linear_spectrum[i])

In [ ]:
linear_spectrum

In [31]:
N = 5

In [43]:
for solution in Trim(leaderboard,linear_spectrum,N):
  print(solution)

RYRDFHTRHSVGRLDNPQDSQGITHGIHSYNAADRFAMAENPGNIMC
GHTDDPDWSFFQMNHFVDQTHGLCANQDAGYSGNLGNFIGDETVRCL
VCTIQFMAKLPFACAWGLMAFTQTYIRMCSNPSKLTRRCHPTLILAE
AEDDFCKLRPSHKNQDEEMDFNICADCIKGIWFQKKWSEANSRYYCH
YPQMKEQNGRQRHDQFWGCRGQSSPVPMCILCDIDKSWYYTDVAFIW
